In [1]:
#env image-processing
import pandas as pd
from sklearn import metrics
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

### Read the data

In [2]:
df=pd.read_csv('../input/winequality_red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [4]:
df.quality.value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

### Make three classes 

In [5]:
quality = df["quality"].values
df["wclass"] = 0

for i, num in enumerate(quality):
    if num <=8 and num >=7:
        df.loc[i,"wclass"] = "high"
    if num <7 and num >5:
        df.loc[i,"wclass"] = "mid"
    if num <=5 and num >=2:
        df.loc[i,"wclass"] = "low"

### Add a column changing  class values to number 

In [6]:
quality = df["wclass"].values
df["wclass_num"] = 0

for i, num in enumerate(quality):
    if num=='high':
        df.loc[i,"wclass_num"] = 2
    if num=='mid':
        df.loc[i,"wclass_num"] = 1
    if num=='low':
        df.loc[i,"wclass_num"] = 0

In [8]:
df.wclass.value_counts()

low     744
mid     638
high    217
Name: wclass, dtype: int64

### Save the data

In [9]:
df.to_csv("../input/winequality.csv", index = False)  

### Look at the data and see 

In [10]:
df1=pd.read_csv("../input/winequality.csv")

In [12]:
df1.head(24)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wclass,wclass_num
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,low,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,low,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,low,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,mid,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,low,0
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,low,0
6,7.9,0.600,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,low,0
7,7.3,0.650,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,high,2
8,7.8,0.580,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,high,2
9,7.5,0.500,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,low,0


In [13]:
df1.wclass.value_counts()

low     744
mid     638
high    217
Name: wclass, dtype: int64

### Create a custom dataset class for pytorch 
- we choose 11 features

In [10]:
#One has to make sure the class index starts at 0, so that we can use cross entropy loss
class WineDataset(Dataset):
    def __init__(self):
        # data loading
        xy = pd.read_csv('../input/winequality.csv')
        self.x = torch.tensor((xy.drop(['quality','wclass','wclass_num'], axis =1)).values).float()
        self.y = torch.tensor((df['wclass_num']-df['wclass_num'].min()).astype('category'))
        self.n_samples = xy.shape[0]
  
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):    
        return self.n_samples
classes =("low","mid","high")

### Look at what the custom dataset returns

In [11]:
dataset = WineDataset()
first_data = dataset[0]
features_label = first_data
print(features_label)

(tensor([ 7.4000,  0.7000,  0.0000,  1.9000,  0.0760, 11.0000, 34.0000,  0.9978,
         3.5100,  0.5600,  9.4000]), tensor(0))


### Create a Dataloader for pytorch 

In [12]:
dataloader = DataLoader(dataset=dataset, batch_size =10, shuffle=False, num_workers=0)
# Make iterator
dataiter = iter(dataloader)
data = dataiter.next()
features, labels = data
#print(features, labels)
print(features.shape)
print(labels)
print([classes[i] for i in labels])

torch.Size([10, 11])
tensor([0, 0, 0, 1, 0, 0, 0, 2, 2, 0])
['low', 'low', 'low', 'mid', 'low', 'low', 'low', 'high', 'high', 'low']


### Create a neural network

In [13]:
class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=11, out_features=100)
        self.fc2 = nn.Linear(in_features=100, out_features=100)
        self.out = nn.Linear(in_features=100, out_features=3)

    
    def forward(self,t):
        t = self.fc1(t)
        t = F.relu(t)
        t = self.fc2(t)
        t = F.relu(t)
        t = self.out(t)
        return t  

### Create functions to qualify the predictions from the model

In [14]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()
def get_balanced_accuracy(preds,labels):
    return metrics.balanced_accuracy_score(labels, preds.argmax(dim=1))

In [15]:
dataloader = DataLoader(dataset=dataset, batch_size =10, shuffle=False, num_workers=0)
# Make iterator
dataiter = iter(dataloader)
data = dataiter.next()
features, labels = data
#print(features, labels)
print(features.shape)
print(labels)

torch.Size([10, 11])
tensor([0, 0, 0, 1, 0, 0, 0, 2, 2, 0])



### Check dimensions of predictions of the nn model

In [16]:
network = Network()
lambda_l2 = 1e-5
learning_rate = 0.0009
pred = network(features)
pred

tensor([[ 0.5651,  0.8059,  0.2516],
        [ 0.8841,  1.4941,  0.3442],
        [ 0.7261,  1.1560,  0.1394],
        [ 0.9581,  1.3081,  0.1478],
        [ 0.5651,  0.8059,  0.2516],
        [ 0.6164,  0.9416,  0.2735],
        [ 0.7815,  1.2345,  0.0733],
        [ 0.5283,  0.7026,  0.4841],
        [ 0.4129,  0.3638,  0.3408],
        [ 1.0491,  2.0302, -0.2231]], grad_fn=<AddmmBackward>)

### All looks good, now run a training loop 

In [17]:
network = Network()
lambda_l2 = 1e-5
learning_rate = 0.0009

train_loader = torch.utils.data.DataLoader(dataset, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(10):
    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        features, labels = batch 
        
        preds = network(features)# Pass Batch

        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights using Adam

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels) 
    
    bal_acc = get_balanced_accuracy(preds,labels)
    
    if epoch >4:
        print("epoch", epoch)
        print(    
            "total_correct:", total_correct,
            "total_pcntg: ",total_correct/1599,
            "loss:", total_loss,
            "balancd_acc",bal_acc
        )

epoch 5
total_correct: 773 total_pcntg:  0.4834271419637273 loss: 16.029676139354706 balancd_acc 0.35772357723577236
epoch 6
total_correct: 753 total_pcntg:  0.4709193245778612 loss: 15.894560635089874 balancd_acc 0.3333333333333333
epoch 7
total_correct: 782 total_pcntg:  0.4890556597873671 loss: 15.893479228019714 balancd_acc 0.35772357723577236
epoch 8
total_correct: 789 total_pcntg:  0.49343339587242024 loss: 15.844582855701447 balancd_acc 0.35772357723577236
epoch 9
total_correct: 807 total_pcntg:  0.5046904315196998 loss: 15.676659166812897 balancd_acc 0.3739837398373984


### This all runs. Now try another model

In [18]:
import random
from IPython import display
from tqdm.notebook import tqdm
seed = 12345
random.seed(seed)
torch.manual_seed(seed)

In [19]:
X = torch.tensor((df.drop(['quality','wclass','wclass_num'], axis =1)).values).float()
y = torch.tensor((df['wclass_num']-df['wclass_num'].min()).astype('category'))

In [20]:
D = 11  # dimensions
C = 3  # num_classes
H1 = 100  # num_hidden_units
H2 = 100  # num_hidden_units
learning_rate = 5e-3
lambda_l2 = 1e-5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
# nn package to create our linear model
# each Linear module has a weight and bias

model = nn.Sequential(
    nn.Linear(D, H1),
    nn.ReLU(),
    nn.Linear(H1, H2),
    nn.ReLU(),
    nn.Linear(H2,C)
)
model.to(device)

# nn package also has different loss functions.
# we use cross entropy loss for our classification task
criterion = torch.nn.CrossEntropyLoss()

# we use the optim package to apply
# ADAM for our parameter updates
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# e = 1.  # plotting purpose

# Training
for t in tqdm(range(100)):
    
    # Feed forward to get the logits
    y_pred = model(X)
    
    # Compute the loss and accuracy
    loss = criterion(y_pred, y)
    score, predicted = torch.max(y_pred, 1)
    acc = (y == predicted).sum().float() / len(y)
    if t>90:
        tqdm.write("[EPOCH]    : %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (t, loss.item(), acc))
    #display.clear_output(wait=True)
    
    # zero the gradients before running
    # the backward pass.
    optimizer.zero_grad()
    
    # Backward pass to compute the gradient
    # of loss w.r.t our learnable params. 
    loss.backward()
    
    # Update params
    optimizer.step()

[EPOCH]    : 91, [LOSS]: 0.771922, [ACCURACY]: 0.624
[EPOCH]    : 92, [LOSS]: 0.772741, [ACCURACY]: 0.629
[EPOCH]    : 93, [LOSS]: 0.776168, [ACCURACY]: 0.630
[EPOCH]    : 94, [LOSS]: 0.774439, [ACCURACY]: 0.613
[EPOCH]    : 95, [LOSS]: 0.765912, [ACCURACY]: 0.635
[EPOCH]    : 96, [LOSS]: 0.761397, [ACCURACY]: 0.627
[EPOCH]    : 97, [LOSS]: 0.763146, [ACCURACY]: 0.629
[EPOCH]    : 98, [LOSS]: 0.765945, [ACCURACY]: 0.639
[EPOCH]    : 99, [LOSS]: 0.767510, [ACCURACY]: 0.616



### This runs too, so we go ahead and write  the python files that are in the src folder

In [22]:
quality_mapping = {
    3: 0,
    4: 1,
    5: 2,
    6: 3,
    7: 4,
    8: 5
}

# you can use the map function of pandas with
# any dictionary to convert the values in a given
# column to values in the dictionary
df.loc[:, "quality"] = df.quality.map(quality_mapping)

In [23]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wclass,wclass_num
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2,low,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,2,low,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,2,low,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,3,mid,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2,low,0


In [24]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'wclass', 'wclass_num'],
      dtype='object')